# Noise (in PyQrack)

Qrack and PyQrack were designed for maximum performance on exact simulation of quantum circuits, hence no noise models are internally supported, yet. Further, PyQrack only supports "pure" quantum states and not density matrix representation, for example. However, there are ways in which we can externally and stochastically apply common types of noise to a PyQrack simulation.

When using external stochastic noise, direct queries of exact quantum state and measurement sampling after a unitary preamble are **not** supported. The methods can still be called, but the pure state returned **cannot** be equivalent to a density matrix with noise, (i.e.: with partially mixed states, in effect,) and sampling operations will sample the happenstance probabilistic noise instead of the full stochastic profile of this noise. To simulate distributions with noise, we must run the circuits top-to-bottom for each sample, when we use probabilistic noise injection methods.

An obvious and simple type of noise injection gadget is for a probabilistic Pauli operator bit flip error.

In [1]:
import random
from pyqrack import Pauli

def inject_pauli_bit_flip_noise(simulator, basis, qubit, probability):
    if (not probability >= 1.) and ((probability <= 0.) or (random.uniform(0., 1.) >= probability)):
        # We avoid the bit flip error
        return

    # We apply a bit flip error
    axis = random.randint(0, 2)
    if basis == Pauli.PauliX:
        simulator.x(qubit)
    elif basis == Pauli.PauliY:
        simulator.y(qubit)
    elif basis == Pauli.PauliZ:
        simulator.z(qubit)

In [2]:
from collections import Counter
from pyqrack import QrackSimulator

qsim = QrackSimulator(1)

results = []
for _ in range(0, 100):
    qsim.reset_all()
    inject_pauli_bit_flip_noise(qsim, Pauli.PauliX, 0, 0.2)
    results.append(qsim.m(0))

print(Counter(results))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Counter({0: 78, 1: 22})


Depolarizing noise (as on a single qubit noise channel) partially replaces a qubit's pure quantum state with a maximally mixed state, according to a tunable noise severity parameter.

(**Warning**: This hypothetical approach to recreating stochastic depolarizing noise has not been vetted. This is purely Dan's first guess, as lead developer of Qrack. It does effectively fully depolarize for a `lam` parameter value of `1.`, though.)

In [3]:
import math

def inject_depolarizing_1qb_noise(simulator, qubit, lam):
    if (lam <= 0.):
        return

    # Original qubit, Z->X basis
    simulator.h(qubit)
    # Random azimuth around Z axis of measurement
    angleZ = random.uniform(0., 4. * math.pi)
    simulator.r(Pauli.PauliZ, angleZ, qubit)
    
    # Allocate an ancilla
    ancilla = simulator.num_qubits()
    simulator.allocate_qubit(ancilla)
    # Partially entangle with the ancilla
    lamAngle = 2. * math.asin(lam ** (1/4))
    simulator.mcr(Pauli.PauliY, lamAngle, [qubit], ancilla)
    # Partially collapse the original state
    simulator.m(ancilla)
    # The ancilla is fully separable, after measurement.
    simulator.release(ancilla)
    
    # Uncompute
    simulator.r(Pauli.PauliZ, -angleZ, qubit)
    simulator.h(qubit)
    
    # Original qubit might be below separability threshold
    simulator.try_separate_1qb(qubit)

In [4]:
qsim = QrackSimulator(2)

results = []
for _ in range(100):
    qsim.reset_all()
    qsim.h(0)
    qsim.mcx([0], 1)
    inject_depolarizing_1qb_noise(qsim, 0, 0.2)
    qsim.mcx([0], 1)
    qsim.h(0)
    results.append(qsim.m_all())

print(Counter(results))

Counter({0: 84, 2: 16})


Can random circuits be simulated more easily, with noise?

In [5]:
import time

def cx(circ, q1, q2):
    circ.mcx([q1], q2)

def cy(circ, q1, q2):
    circ.mcy([q1], q2)

def cz(circ, q1, q2):
    circ.mcz([q1], q2)

def acx(circ, q1, q2):
    circ.macx([q1], q2)

def acy(circ, q1, q2):
    circ.macy([q1], q2)

def acz(circ, q1, q2):
    circ.macz([q1], q2)

def swap(circ, q1, q2):
    circ.swap(q1, q2)

def random_circuit(circ, depth, noise):
    circ.reset_all()
    num_qubits = circ.num_qubits()
    one_bit_gates = circ.x, circ.y, circ.z, circ.h, circ.s, circ.adjs, circ.t, circ.adjt
    two_bit_gates = swap, cx, cz, cy, acx, acz, acy
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    colLen = math.floor(math.sqrt(num_qubits))
    while ((math.floor(num_qubits / colLen) * colLen) != num_qubits):
        colLen = colLen - 1
    rowLen = num_qubits // colLen;

    start = time.time()

    for i in range(depth):
        # Single bit gates
        for j in range(num_qubits):
            g = random.choice(one_bit_gates)
            g(j)
            # Add noise channel for non-Clifford gates.
            # See https://arxiv.org/abs/1810.03176 for suggestion of only non-Clifford noise.
            if g == circ.t or g == circ.adjt:
                inject_depolarizing_1qb_noise(circ, j, noise)

        gate = gateSequence[0]
        gateSequence.pop(0)
        gateSequence.append(gate)

        for row in range(1, rowLen, 2):
            for col in range(0, colLen):
                tempRow = row;
                tempCol = col;

                tempRow = tempRow + (1 if (gate & 2) else -1)
                if colLen != 1:
                    tempCol = tempCol + (1 if (gate & 1) else 0)

                if (tempRow < 0) or (tempCol < 0) or (tempRow >= rowLen) or (tempCol >= colLen):
                    continue;

                b1 = row * colLen + col;
                b2 = tempRow * colLen + tempCol;

                # Two bit gates
                g = random.choice(two_bit_gates)
                g(circ, b1, b2)

    circ.m_all()

    return time.time() - start

In [6]:
import os

# For large simulations, setting the max paging qubit count correctly prevents execution from hanging.
#  (This is the maximum "naive" state vector simulation you can do only with distributed "QPager" layer.)
os.environ["QRACK_MAX_PAGING_QB"] = "30"
# The separability threshold introduces a kind of non-physical noise, which Qrack users call "hyperpolarizing,"
# but it's opportune for simulation  performance, due to increased Schmidt decomposition.
os.environ["QRACK_QUNIT_SEPARABILITY_THRESHOLD"] = "0.0002"
# os.environ.pop('QRACK_QUNIT_SEPARABILITY_THRESHOLD', None)

# 36 qubits
num_qubits = 36
depth = 8
noise = 0.0001
qsim = QrackSimulator(num_qubits)

trialCount = 100
failureCount = 0
times = []
for _ in range(trialCount):
    try:
        times.append(random_circuit(qsim, depth, noise))
    except:
        failureCount = failureCount + 1
print("Noisy simulation, success fraction: " + str((trialCount - failureCount) / trialCount))
if len(times) == 0:
    print("All trials failed!")
else:
    print("Noisy simulation, average seconds: " + str(sum(times) / len(times)))

Noisy simulation, success fraction: 1.0
Noisy simulation, average seconds: 0.029455409049987794


We incur the overhead of simulating the noise channels themselves, to start. However, if we can increase Schmidt decomposition by reducing entanglement with noise, we might lower our memory footprint.